In [1]:
from pathlib import Path
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
import numpy as np

# Ruta
BASE = Path.cwd().parent
PATH_CSV = BASE / "DB_Model" / "df_modelo.csv"
PATH_SUM = BASE / "Resumen_de_variables_del_modelo__depurado_.csv" 

# Lectura
df = pd.read_csv(PATH_CSV, engine="python", sep=None)

# Resumen
print(f"Filas: {len(df):,} | Columnas: {len(df.columns)}")
print("Columnas:")
print(list(df.columns))

display(df.head(10))

Filas: 72,292 | Columnas: 44
Columnas:
['\ufeffLlave2', 'Nombre_linea', 'Fecha_aprobacion', 'Antiguedad_meses', 'Nombre_fondo', 'Valor_financiacion', 'Cuotas', 'Tipo_interes', 'Vr_neto_matricula', 'Y_categorica', 'IDBANNER', 'FECHA_NACIMIENTO', 'ESTADO_CIVIL', 'GENERO', 'FACULTAD', 'PROGRAMA', 'NIVEL', 'ESTADO', 'TIPOESTUDIANTE', 'SEDE', 'SELLO', 'CARGA', 'GRUPO_ETNICO', 'TIPO_DISCAPACIDAD', 'NACIONALIDAD', 'Mora', 'Valor_cuota_inicial', 'Valor_primera_cuota', 'fecha de pago', 'Validación valor financiado', 'Detalle estado final', 'Tipo_estudiante', 'Operación', 'Cate', 'Subcate', 'Cohorte', 'Mes', 'Cliente', 'Media Score', 'AñoB', 'VALOR MAXIMO', 'VALOR MEDIO', 'VALOR BAJO', 'Plataforma']


,﻿Llave2,Nombre_linea,Fecha_aprobacion,Antiguedad_meses,Nombre_fondo,Valor_financiacion,Cuotas,Tipo_interes,Vr_neto_matricula,Y_categorica,...,Subcate,Cohorte,Mes,Cliente,Media Score,AñoB,VALOR MAXIMO,VALOR MEDIO,VALOR BAJO,Plataforma
0,100046415_202110_13110,ESTUDIA YA-TECH GLOBAL,12/02/2021,56,BOGOTA,1895042,12,Cap. Fijo-Int. vencido,2190800.0,Bajo,...,Tiene Sello Financiero (Aprobado Directo),Cohorte 1,Febrero,Estudiante,Scoring = o > 721,2021.0,0.0,0.0,0.0,ROMBO V1
1,100094861_202131_13118,ESTUDIA YA-TECH GLOBAL,15/02/2021,56,CONVENIO SENA NACIONAL,1312400,4,Cap. Fijo-Int. vencido,1544000.0,Bajo,...,Tiene Sello Financiero (Aprobado Directo),Cohorte 1,Febrero,Estudiante,Scoring = o > 721,2021.0,0.0,0.0,0.0,ROMBO V1
2,100090085_202141_13292,ESTUDIA YA-TECH GLOBAL LINEA BLANCA,5/02/2021,56,CONVENIO INESUR,2589525,4,Cap. Fijo-Int. vencido,3385000.0,Bajo,...,Tiene Sello Financiero (Lista Blanca),Cohorte 1,Febrero,Estudiante,Scoring = o > 401 y < o = 480,2021.0,0.0,0.0,0.0,ROMBO V1
3,100082960_202131_13165,ESTUDIA YA-TECH GLOBAL,25/02/2021,56,CONVENIO CONSTRUYENDO FUTURO - CONSFUTUR S.A.S,1307300,6,Cap. Fijo-Int. vencido,1538000.0,Bajo,...,Tiene Sello Financiero (Aprobado Directo),Cohorte 1,Febrero,Estudiante,Scoring = o > 561 y < o = 639,2021.0,0.0,0.0,0.0,ROMBO V1
4,100083999_202110_13175,ESTUDIA YA-TECH GLOBAL,15/02/2021,56,BOGOTA,2186720,6,Cap. Fijo-Int. vencido,2528000.0,Bajo,...,Tiene Sello Financiero (Aprobado Directo),Cohorte 1,Febrero,Estudiante,Scoring = o > 561 y < o = 639,2021.0,0.0,0.0,0.0,ROMBO V1
5,100039484_202151_13183,ESTUDIA YA-TECH GLOBAL,15/02/2021,56,BOGOTA,2906250,12,Cap. Fijo-Int. vencido,3419250.0,Bajo,...,Tiene Sello Financiero (Aprobado Directo),Cohorte 1,Febrero,Estudiante,Scoring = o > 721,2021.0,0.0,0.0,0.0,ROMBO V1
6,100095400_202131_13154,ESTUDIA YA-TECH GLOBAL,15/02/2021,56,BOGOTA,1147457,6,Cap. Fijo-Int. vencido,1421000.0,Bajo,...,Aprobado (Aprobado Directo),Cohorte 1,Febrero,Estudiante,Scoring = o > 640 y < o = 720,2021.0,0.0,0.0,0.0,ROMBO V1
7,100074911_202141_13275,ESTUDIA YA-TECH GLOBAL,15/02/2021,56,BOGOTA,1242300,4,Cap. Fijo-Int. vencido,1462300.0,Bajo,...,Tiene Sello Financiero (Aprobado Directo),Cohorte 1,Febrero,Estudiante,Scoring = o > 640 y < o = 720,2021.0,0.0,0.0,0.0,ROMBO V1
8,100077506_202151_13347,ESTUDIA YA-TECH GLOBAL,5/02/2021,56,CONVENIO FUNDACIÓN YANAPAX,3245960,12,Cap. Fijo-Int. vencido,3819960.0,Bajo,...,Tiene Sello Financiero (Aprobado Directo),Cohorte 1,Febrero,Estudiante,Scoring = o > 640 y < o = 720,2021.0,0.0,0.0,0.0,ROMBO V1
9,100070822_202151_13369,ESTUDIA YA-TECH GLOBAL LINEA BLANCA,5/02/2021,56,CONVENIO FUNDACIÓN YANAPAX,3476500,4,Cap. Fijo-Int. vencido,4090000.0,Bajo,...,Tiene Sello Financiero (Lista Blanca),Cohorte 1,Febrero,Estudiante,Scoring = o > 640 y < o = 720,2021.0,0.0,0.0,0.0,ROMBO V1


In [2]:
# Normaliza encabezados 
df = df.copy()
df.columns = (
    df.columns.astype(str)
      .str.replace('\ufeff', '', regex=False)  
      .str.strip()                             
)

# 1) Separar Y y X
TARGET = "Y_categorica"
DROP = ["Llave2", "Nombre_linea", "IDBANNER"]
y = df[TARGET].copy()
X = df.drop([TARGET] + [c for c in DROP if c in df.columns], axis=1).copy()

# 2) Columnas numéricas / categóricas
num_cols = [c for c in X.columns if pd.api.types.is_numeric_dtype(X[c])]
cat_cols = [c for c in X.columns if c not in num_cols]

# 3) Preprocesamiento 
num_tf = SimpleImputer(strategy="median")
cat_tf = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="most_frequent")),
    ("onehot", OneHotEncoder(
        handle_unknown="ignore",
        sparse_output=True,          
        min_frequency=0.01,       
        dtype=np.float32         
    ))
])
preprocess = ColumnTransformer(
    transformers=[
        ("num", num_tf, num_cols),
        ("cat", cat_tf, cat_cols),
    ],
    remainder="drop",
    sparse_threshold=1.0  
)

# 4) Partición
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.20, random_state=42, stratify=y
)

# Validación del preprocesador
_ = preprocess.fit(X_train, y_train)

print("X:", X.shape, "| y:", y.shape)
print("Numéricas:", len(num_cols), "| Categóricas:", len(cat_cols))
print("Train:", X_train.shape, "Test:", X_test.shape)

X: (72292, 40) | y: (72292,)
Numéricas: 10 | Categóricas: 30
Train: (57833, 40) Test: (14459, 40)


In [3]:
#LightGBM + RandomizedSearchCV 

from tempfile import mkdtemp
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OrdinalEncoder
from sklearn.impute import SimpleImputer
from sklearn.model_selection import RandomizedSearchCV, StratifiedKFold
from sklearn.metrics import roc_auc_score, accuracy_score, precision_score, recall_score, classification_report
from sklearn.preprocessing import LabelEncoder
from scipy.stats import randint, uniform
from lightgbm import LGBMClassifier
import numpy as np
import pandas as pd

le = LabelEncoder().fit(y_train)
y_train_enc = le.transform(y_train)
y_test_enc  = le.transform(y_test)
n_clases = len(le.classes_)

# 1) Columnas
num_cols = [c for c in X_train.columns if pd.api.types.is_numeric_dtype(X_train[c])]
cat_cols = [c for c in X_train.columns if c not in num_cols]

# 2) Prepro 
lgb_prep = ColumnTransformer(
    transformers=[
        ("num", SimpleImputer(strategy="median"), num_cols),
        ("cat", Pipeline([
            ("imp", SimpleImputer(strategy="most_frequent")),
            ("ord", OrdinalEncoder(handle_unknown="use_encoded_value", unknown_value=-1))
        ]), cat_cols),
    ],
    remainder="drop"
)

# 3) Modelo
lgb = LGBMClassifier(
    objective="multiclass",
    num_class=n_clases,
    metric="multi_logloss",
    n_jobs=-1,
    random_state=42
)

pipe = Pipeline([("prep", lgb_prep), ("clf", lgb)])
pipe.set_params(memory=mkdtemp()) 

# 4) Búsqueda aleatoria 
param_dist = {
    "clf__n_estimators": randint(300, 900),
    "clf__learning_rate": uniform(0.02, 0.18),
    "clf__num_leaves": randint(15, 63),
    "clf__max_depth": randint(3, 12),
    "clf__min_child_samples": randint(10, 60),
    "clf__subsample": uniform(0.6, 0.4),        
    "clf__colsample_bytree": uniform(0.6, 0.4), 
    "clf__reg_alpha": uniform(0.0, 0.5),
    "clf__reg_lambda": uniform(0.0, 0.5),
}

cv = StratifiedKFold(n_splits=3, shuffle=True, random_state=42)
rs = RandomizedSearchCV(
    estimator=pipe,
    param_distributions=param_dist,
    n_iter=25,
    scoring="roc_auc_ovr",    
    cv=cv,
    n_jobs=-1,
    verbose=1,
    random_state=42
)

rs.fit(X_train, y_train_enc)
best_model = rs.best_estimator_
cv_auc = rs.best_score_

# 5) Evaluación
proba_test = best_model.predict_proba(X_test)           
test_auc   = roc_auc_score(y_test_enc, proba_test, multi_class="ovr", average="macro")

y_pred_enc = best_model.predict(X_test)
y_pred     = le.inverse_transform(y_pred_enc)                

acc  = accuracy_score(y_test, y_pred)
prec = precision_score(y_test, y_pred, average="macro", zero_division=0)
rec  = recall_score(y_test, y_pred, average="macro", zero_division=0)

print("Mejor modelo LightGBM (RandomizedSearchCV)")
print("Mejores params:", rs.best_params_)
print("Label mapping:", {int(i): cls for i, cls in enumerate(le.classes_)})
print(f"CV AUC (OVR):   {cv_auc:0.4f}")
print(f"Test AUC (OVR): {test_auc:0.4f}")
print(f"Accuracy: {acc:0.4f} | Precision(macro): {prec:0.4f} | Recall(macro): {rec:0.4f}")
print("\nReporte por clase:\n", classification_report(y_test, y_pred, zero_division=0))


Fitting 3 folds for each of 25 candidates, totalling 75 fits
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.016051 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 2993
[LightGBM] [Info] Number of data points in the train set: 57833, number of used features: 40
[LightGBM] [Info] Start training from score -2.283794
[LightGBM] [Info] Start training from score -0.115066
[LightGBM] [Info] Start training from score -4.991505
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with posit

c:\Users\Juanr\anaconda3_\envs\mi_entorno\lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(
c:\Users\Juanr\anaconda3_\envs\mi_entorno\lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(


Mejor modelo LightGBM (RandomizedSearchCV)
Mejores params: {'clf__colsample_bytree': np.float64(0.798070764044508), 'clf__learning_rate': np.float64(0.02618993380073931), 'clf__max_depth': 3, 'clf__min_child_samples': 45, 'clf__n_estimators': 861, 'clf__num_leaves': 54, 'clf__reg_alpha': np.float64(0.37768070515882624), 'clf__reg_lambda': np.float64(0.21257793724562235), 'clf__subsample': np.float64(0.6831766651472755)}
Label mapping: {0: 'Alto', 1: 'Bajo', 2: 'Medio'}
CV AUC (OVR):   0.8282
Test AUC (OVR): 0.8413
Accuracy: 0.8910 | Precision(macro): 0.4620 | Recall(macro): 0.3402

Reporte por clase:
               precision    recall  f1-score   support

        Alto       0.49      0.02      0.05      1473
        Bajo       0.89      1.00      0.94     12887
       Medio       0.00      0.00      0.00        99

    accuracy                           0.89     14459
   macro avg       0.46      0.34      0.33     14459
weighted avg       0.85      0.89      0.84     14459



In [ ]:
#Mejor modelo LightGBM
#Mejores params: {'clf__colsample_bytree': np.float64(0.798070764044508), 'clf__learning_rate': np.float64(0.02618993380073931), 'clf__max_depth': 3, 'clf__min_child_samples': 45, 'clf__n_estimators': 861, 'clf__num_leaves': 54, 'clf__reg_alpha': np.float64(0.37768070515882624), 'clf__reg_lambda': np.float64(0.21257793724562235), 'clf__subsample': np.float64(0.6831766651472755)}

In [4]:
# Exportar predicciones y unirlas a la base original 

from pathlib import Path
import pandas as pd
import numpy as np

BASE = Path.cwd().parent
OUT_CSV = BASE / "DB_Model" / "df_con_predicciones_TEST_LGBM.csv"

# Probabilidades y clases 
proba  = best_model.predict_proba(X_test)
clases_cod = best_model.named_steps["clf"].classes_     
clases_nom = le.inverse_transform(clases_cod)            

# Predicción de clase 
y_hat = le.inverse_transform(best_model.predict(X_test))

# DataFrame de predicciones 
preds = pd.DataFrame(index=X_test.index)
preds["y_true"] = y_test.values
preds["y_pred"] = y_hat
for i, cls_name in enumerate(clases_nom):
    preds[f"proba_{str(cls_name)}"] = proba[:, i].astype(np.float32)

# 1) SOLO filas de test con TODAS las columnas originales + predicciones
df_test_con_preds = df.loc[X_test.index].join(preds)
df_test_con_preds.to_csv(OUT_CSV, index=True)
print(f"Guardado: {OUT_CSV}")
print(df_test_con_preds.head(5))

# 2) BASE COMPLETA con predicciones 
df_full_con_preds = df.join(preds, how="left")
df_full_con_preds.to_csv(BASE / "DB_Model" / "df_con_predicciones_FULL_LGBM.csv", index=True)


c:\Users\Juanr\anaconda3_\envs\mi_entorno\lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(
c:\Users\Juanr\anaconda3_\envs\mi_entorno\lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(


Guardado: c:\Users\Juanr\Downloads\Prediccion_Morosidad_Estudiantil_\Prediccion_Morosidad_Estudiantil\DB_Model\df_con_predicciones_TEST_LGBM.csv
                      ﻿Llave2                           Nombre_linea  \
13618  100105637_202120_15481                 ESTUDIA YA-TECH GLOBAL   
11456  100124843_202243_20266  ESTUDIA YA-TECH GLOBAL PROMO NAVIDEÑA   
416    100113510_202304_25761             ESTUDIA YA-TECH GLOBAL 1.8   
51557  100222334_202553_82758               LINEA CRÉDITO CUOTA FIJO   
16315  100157683_202320_34444             ESTUDIA YA-TECH GLOBAL 1.8   

      Fecha_aprobacion  Antiguedad_meses Nombre_fondo  Valor_financiacion  \
13618       30/08/2021                50       BOGOTA             1979650   
11456       15/06/2022                40       BOGOTA             2689543   
416         30/11/2022                35       BOGOTA             1275000   
51557       20/07/2025                 3       BOGOTA             4023050   
16315       20/07/2023               